In [36]:
# Descargar script
!jupyter nbconvert --to python buscar_productos.ipynb

[NbConvertApp] Converting notebook buscar_productos.ipynb to python
[NbConvertApp] Writing 5559 bytes to buscar_productos.py


In [2]:
# Import data
import spacy
from _general import clean_text, load_data
import pandas as pd
from annoy import AnnoyIndex
import json
from cryptography.fernet import Fernet

In [3]:
# BUSCAR N NUMERO DE PRODUCTOS MAS PARECIDOS

def serach_product(word):
    """ 
    Takes: str
    Returns: list of tuples (TRAZA, nombre de producto)
    """
    # Load model
    nlp = spacy.load("./model")

    # Load index
    annoy_index = AnnoyIndex(500, 'euclidean')
    annoy_index.load('./data/product_index.ann')
    
    # Find vector of word based in spacy model
    word = clean_text(word,lowerCase=True,unidecodeStandard=True,skipStopWords=True)
    search_doc = nlp(word)
    search_vector = search_doc.vector

    # Search in index the 10 closest points to the vector found
    res = annoy_index.get_nns_by_vector(search_vector, 10)

    # Get the word that is represented by each of the vectors found
    # Get values from reference map (TRAZA, nombre de producto)
    with open("./data/reference_map.json", "r", encoding="utf-8") as f:
        reference = json.load(f)

    productos = []
    for ix in res:
        p = reference[str(ix)]
        productos.append(p)



    return productos
        

In [4]:
results = serach_product("silla")
print(results)

[['163cee6d', 'silla'], ['6531f72e', 'silla gerencial'], ['db33e64a', 'silla ejecutiva'], ['6a268d44', 'silla amy'], ['eb0d8405', 'silla prisma'], ['86c53cf2', 'silla rudy'], ['79c3d619', 'silla rudy'], ['79a003bb', 'silla rudy'], ['ee3fc80a', 'silla rudy lpt'], ['41992973', 'silla rudy ergonomica']]


In [32]:
# Load past data
def open_productos_cotizados():
    # Load encrypted data
    with open('./data/filekey.key', 'rb') as filekey:
        encrypt_key = filekey.read()

    # Create decoder
    f = Fernet(encrypt_key)

    token = pd.read_csv('./data/productos.csv') 
    token = token.applymap(lambda x: bytes(x[2:-1],'utf-8'))
    token  = token.applymap(lambda x: f.decrypt(x))
    token = token.applymap(lambda x: x.decode('utf-8'))

    columns = list(token.columns.values)
    columns = map(lambda x: bytes(x[2:-1],'utf-8'),columns)
    columns = map(lambda x: f.decrypt(x), columns)
    columns = map(lambda x: x.decode('utf-8'), columns)


    token.columns = columns
    productos_cotizados = token
    productos_cotizados.set_index('TRAZA',inplace=True)

    return productos_cotizados
    

In [33]:
productos_cotizados = open_productos_cotizados()
productos_cotizados

,Cotizacion,Producto Solicitado,Producto Ofrecido,U. Medida,Cantidad,Costo x Unidad,Costo x Unidad SIN IGV en Soles,Precio Venta x Unidad,Link,Observaciones,Habilitar y Deshabilitar,Adjunto,Adjunto Ficha Tecnica,Imagen Referencial,Ultima revisión,OC a proveedor
TRAZA,,,,,,,,,,,,,,,,
152e43f1,47630ddf50e6f0e4,5a76b72b,"MONITOR LG 21.5’’ (22MN430M-B), 1920 X 1080 (F...",Unidad,2,550.5,466.53,601.17,,,Deshabilitado,,,,,None
1ac5849a,47630ddf50e6f0e4,3d56f01b,TECLADO INALAMBRICO + MOUSE INALAMBRICO GENIU...,Unidad,15,78.4,66.44,82.1,,,Deshabilitado,,,,,None
d79583f9,47630ddfb5351092,3d56f01b,TECLADO INALAMBRICO + MOUSE INALAMBRICO TEROS...,Unidad,15,59.9,50.76,82.1,https://dasmitec.pe/productos/kit-inalambrico-...,,Deshabilitado,,,,,None
aedc03a3,47630ddf05708cfd,3d56f01b,COMBO LOGITECH: TECLADO + MOUSE MK220 WIRELESS...,Unidad,15,77.5,77.5,82.1,https://www.memorykings.com.pe/producto/327993...,,Deshabilitado,,,,,None
398d553b,47630ddf05708cfd,5a76b72b,MONITOR 22” SAMSUNG LF22T350FHLXPE iPS FHD HDM...,Unidad,2,567.5,567.5,601.17,,,Habilitado,,,,,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
a6443415,f1ab26fa,97df0ae1,PLANCHA DE EMPASTAR M/GOMAAMARILLO WINGS 30 cm,Unidad,3,10.38,8.7966,0,,,Deshabilitado,,,,,None
5db96e92,f1ab26fa,97df0ae1,PLANCHA DE EMPASTA M/GOMAEUROTOLLS 30 cm,Unidad,3,9.51,8.0593,0,,,Deshabilitado,,,,,None
f70115b3,f1ab26fa,97df0ae1,PLANCHA DE EMPASTAR M/GOMA ROJA PROFIELD 30 CM,Unidad,3,10.7,9.0678,0,,,Deshabilitado,,,,,None


In [34]:
# BUSCAR PRODUCTOS IMPORTADOS CON PLANILLA 
def to_str(x):
    x = str(x)
    if x == 'nan':
        return ""
    else:
        return x
    
def search_file(path, productos_cotizados):

    # Leer datos subidos por usuario
    planilla = pd.read_excel(path, index_col= None)
    productos_solicitados  = planilla[["Producto Solicitado",
                                       "Nombre de producto Solicitado",
                                       "U. Medida solicitada", 
                                       "Cantidad Solicitada"]].values.tolist() 

    # Create new dataframe to save n products found for each product requested
    todos_productos_similares = pd.DataFrame()

    # Search every product input by the user
    for TRAZA_sol, prod, medida, cant in productos_solicitados:
        productos_similares = serach_product(prod) # [[producto_TRAZA, producto_normalized_name],...[]]
        
        # Get all trazas
        trazas = []
        for (traza, _) in productos_similares:
            
            trazas.append(str(traza))
        
        # Find products from FIGsac database
        productos_similares = productos_cotizados.loc[trazas]
    
        # Añadir info de producto_solicitado a productos_similares
        productos_similares["Producto Solicitado"] =  TRAZA_sol
        productos_similares["Nombre de producto Solicitado"] =  prod
        productos_similares["U. Medida solicitada"] =  medida
        productos_similares["Cantidad Solicitada"] =  cant

        # Add all productos_similares to todos_productos_similares
        productos_similares.reset_index(inplace=True)
        todos_productos_similares =  pd.concat([todos_productos_similares, productos_similares], ignore_index = True)
    
    # Add comentaries about the origin of data
    todos_productos_similares["Observaciones"] = "Datos copiados de producto " + todos_productos_similares["TRAZA"].map(str)  + todos_productos_similares["Observaciones"].map(to_str) 

    # Copy only necessary data to return
    resultado = pd.DataFrame()
    resultado["Cotizacion"] = todos_productos_similares["Cotizacion"]
    resultado["Producto Solicitado"] = todos_productos_similares["Producto Solicitado"]
    resultado["Nombre de producto Solicitado"] = todos_productos_similares["Nombre de producto Solicitado"]
    resultado["U. Medida solicitada"] = todos_productos_similares["U. Medida solicitada"]
    resultado["Cantidad Solicitada"] = todos_productos_similares["Cantidad Solicitada"]
    resultado["Producto Ofrecido"] = todos_productos_similares["Producto Ofrecido"]
    resultado["U. Medida"] = todos_productos_similares["U. Medida"]
    resultado["Cantidad"] = todos_productos_similares["Cantidad"]
    resultado["Costo x Unidad"] = todos_productos_similares["Costo x Unidad"]
    resultado["Precio Venta x Unidad"] = todos_productos_similares["Precio Venta x Unidad"]
    resultado["Cotizacion"] = todos_productos_similares["Cotizacion"]
    resultado["Observaciones"] = todos_productos_similares["Observaciones"]  

    return resultado
            


In [35]:
path = "./secret/Base de datos_Carga masiva_Datos Generales del Proceso--R2023-505PlanillaInfo1.xlsx"
resultados_trazas = search_file(path, productos_cotizados)
resultados_trazas

,Cotizacion,Producto Solicitado,Nombre de producto Solicitado,U. Medida solicitada,Cantidad Solicitada,Producto Ofrecido,U. Medida,Cantidad,Costo x Unidad,Precio Venta x Unidad,Observaciones
0,968f9383610f42b6,e534d2ce,Sierra Copa 22cm,Unidad,2,MG MUG CERAMICA Alto 14.5 cm Diámetro 6.2 cm C...,Unidad,0,11.13,16.7,Datos copiados de producto f214a4cc
1,2a64b4f5,e534d2ce,Sierra Copa 22cm,Unidad,2,"Dispensador de jabón Orange Aura. 14cm alto, ...",Unidad,6,21.9,28.3,Datos copiados de producto ce4bfbaa
2,2a64b4f5,e534d2ce,Sierra Copa 22cm,Unidad,2,Dispensador de jabón Granito Bl de cemento mod...,Unidad,10,26.9,28.3,Datos copiados de producto eaf0f72a
3,2c0831c5bbc49183,e534d2ce,Sierra Copa 22cm,Unidad,2,DISPENSADOR DE JABÓN LÍQUIDO O GEL ALCOHOL ACE...,Unidad,4,86,109.32,Datos copiados de producto a9da79bd
4,968f9383610f42b6,e534d2ce,Sierra Copa 22cm,Unidad,2,MG TAZA CON CUCHARA 450 ML C/TAPA Y CUCHARA MA...,Unidad,0,10.5,15.75,Datos copiados de producto f3804e48
...,...,...,...,...,...,...,...,...,...,...,...
65,8523944c,a59cba72,"Espátula metálica 4"" c/mango de madera Tramont...",Unidad,20,"Espátula Tolsen Mango de Madera 3"" 4004",Unidad,15,1.66,4.98,Datos copiados de producto c9e49d05ver link
66,41a12cbf,a59cba72,"Espátula metálica 4"" c/mango de madera Tramont...",Unidad,20,"Espátula Tolsen Mango de Madera 3"" 40004",Unidad,15,1.66,4.98,Datos copiados de producto c9e49d07ver link
67,9ac9ffbe,a59cba72,"Espátula metálica 4"" c/mango de madera Tramont...",Unidad,20,"Espátula Tolsen Mango de Madera 3"" 80MM MET...",Unidad,15,7,9.85,Datos copiados de producto 34e202c7
68,7b7dfb0dg4ff22ac,a59cba72,"Espátula metálica 4"" c/mango de madera Tramont...",Unidad,20,"Espátula fabricada en acero 3"". Mango de mader...",Unidad,5,5.9,6.75,Datos copiados de producto 1cd78521
